# FIT5196 Assessment 1 Task 2
#### Student Name:Balaji Ippagunta
#### Student ID: 29876451

Date: 15/04/2017

Version: 2.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

Libraries used:
* docx (for parsing docx document)
* re (for regular expression) 
* nltk (Natural Language Toolkit,)
* nltk.collocations (for finding bigrams)
* nltk.tokenize (for tokenization)



## 1. Introduction
The task is to extract the unit data from the doucment and create a vocabulary.
The final output of the file is vocabulary text file which has all the tokens after stemming and a countVec file which gives the sparse representation of the units
More details for each task will be given in the following sections.

In [1]:
from docx import Document
import re
from itertools import chain
from nltk.tokenize import RegexpTokenizer 
from nltk.tokenize import MWETokenizer
import nltk

## 2 Loading and transformation fo the data 

Data is read from the doc file and the last two columns are clubbed to gether and are saved in dictionary with value in the first column as key 

There are multiple tables in the document we are going to iterate over each table and each row in the table extract all the cells in each row for extracting the data.
We are using the variable counter to keep track of the varibale's cell number for using the variable asa key and also to normalise the case sensitivity as per the specification

In [2]:
doc = Document('29876451.docx')
unitdict={}
for table in doc.tables:
    for row in table.rows:
        counter=0
        contentList=[]
        contentString=""
        for cell in row.cells:
            if counter==0:
                key=cell.text
                counter+=1
            elif counter==1:
                cell.text=cell.text[0].lower() + cell.text[1:]
                contentString=contentString+" "+cell.text
                counter+=1
            elif counter==2:
                outcomesstr=""
                temp=cell.text.split(', \'')
                for ele in temp:
                    i=0
                    # We are converting the case of the first alphabet in the scentence to a lower case
                    while i is not "change":
                        if ele[i].isalpha():
                            if i==0:
                                ele=ele[i].lower()+ele[i+1:]
                            else:
                                ele=ele[0:i-1]+ele[i].lower()+ele[i+1:]
                            i="change"
                                    
                        else:
                            i+=1
                    outcomesstr=" "+ele
                    contentString=contentString+" "+outcomesstr
                unitdict[key]=contentString
                contentList=[]
                counter=0
#Removing the data with Title as key as they are the headings of the table columns 
unitdict.pop('Title', None)

' synopsis  outcomes'

## 3 Tokenization

Function for tokenizing the words in the document with the given regular expression

In [3]:
def tokenizeunitdetails(unitid):
     
    tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?")
    tokens = tokenizer.tokenize(unitdict[unitid])
    return (unitid, tokens) # return a tupel of unitid and a list of tokens

units_tokenized = dict(tokenizeunitdetails(unitid) for unitid in unitdict.keys())

We are now tokenizing the scentences by iterating over the dictionary and are saving the tokens into a new dictionary with unit id as key

In [4]:
for key,value in units_tokenized.items():
    for element in value:
        if not (element.isalpha()):
            value.remove(element)

In [5]:
combinedtokensList = list(chain.from_iterable(units_tokenized.values()))

Let's check how many unique words we have in the whole file and the lexical diversity (i.e., the average number of times a type apprearing in the collection.)

In [6]:
vocab = set(combinedtokensList)
lexical_diversity = len(combinedtokensList)/len(vocab)
print ("Vocabulary size: ",len(vocab),"\nTotal number of tokens: ", len(combinedtokensList), \
"\nLexical diversity: ", lexical_diversity)

Vocabulary size:  4000 
Total number of tokens:  30841 
Lexical diversity:  7.71025


## 4 Stop Words

Removing stop words from the tokens as per the specification

In [7]:
stopwords = []
with open('./stopwords_en.txt') as f:
    stopwords = f.read().splitlines()
stopwords_set = set(stopwords)
stopped_tokens = [w for w in combinedtokensList if w not in stopwords_set]

Removing tokens which are less than 3 letters length and also removing the tokens from the dictionary which are in the stop words list

In [8]:
for unitid,content in units_tokenized.items():
    for token in content:
        if token in stopwords_set:
            content.remove(token)
        
for unitid,content in units_tokenized.items():
    for token in content:
        if len(token)<3:
            content.remove(token)
            
for ele in stopped_tokens:
    if len(ele)<3:
            stopped_tokens.remove(ele)

## 5 Bigrams

Generating the top 200 Bigrams for the given document

In [9]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(stopped_tokens)
bigram_finder.apply_freq_filter(20)
bigram_finder.apply_word_filter(lambda w: len(w) < 3)# or w.lower() in ignored_words)
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) # Top-100 bigrams

## 6 Retokenizing with bigrams

In [10]:
mwetokenizer = MWETokenizer(top_200_bigrams)
colloc_units =  dict((unitid, mwetokenizer.tokenize(content)) for unitid,content in units_tokenized.items())
all_words_colloc = list(chain.from_iterable(colloc_units.values()))

In [11]:
for value in all_words_colloc:
    if len(value)<3:
        all_words_colloc.remove(value)
        #print(value)
colloc_voc = list(set(all_words_colloc))

Calculating document frequency for all the tokens

In [12]:
freq_dict={}
for token in all_words_colloc:
    freq_dict[token]=0
for token in colloc_voc:
    for key,value in units_tokenized.items():
        if token in value:
            freq_dict[token]+=1

Identifying the tokens which are with less than 5% frequency and more than 95% frequency

In [13]:
lessthan5=[]
morethan95=[]
for key,value in freq_dict.items():
    if value<10:
        lessthan5.append(key)
    elif value >184:
        morethan95.append(key)

Deleting the tokens which are with less than 5% frequency and more than 95% frequency

In [14]:
for token in all_words_colloc:
    if token in morethan95:
        all_words_colloc.remove(token)
    elif token in lessthan5:
        all_words_colloc.remove(token)
final_tokens_set=list(set(all_words_colloc))
final_tokens_set.sort()

## 7 Stemming

Stemming the words with the help of Porter Stemmer

In [15]:
stemmer = nltk.PorterStemmer()
final_tokens =[]
final_tokens = final_tokens + [stemmer.stem(w) for w in final_tokens_set ]
final_tokens.sort()
final_tokens=list(set(final_tokens))
print(len(final_tokens))

1625


Creating the vocab.txt file

In [16]:
integer_index=1
vocab_dict={}
vocabfile = open("29876451_vocab.txt", "a")
for value in final_tokens:
    vocabfile.write(value+":"+str(integer_index)+"\n")
    vocab_dict[value]=str(integer_index)
    integer_index+=1
vocabfile.close()
print("29876451_vocab.txt File generated")

29876451_vocab.txt File generated


Creating the countVec.txt file

In [17]:

vectorfileInput=""
for key,value in colloc_units.items():
    #print("1")
    variable=""
    for token in final_tokens:
        count=0
        #print("1")
        for item in value:
            if token==item:
                count+=1
        if count>0:
            attribute=vocab_dict[token]
            variable=variable+str(attribute)+":"+str(count)+" ,"
    variable=variable[0:len(variable)-1]
    vectorfileInput=vectorfileInput+key+", "+variable+"\n"
    #print(key+", "+variable)
vectorfile = open("29876451_countVec.txt", "a")
vectorfile.write(""+vectorfileInput)
vectorfile.close()
print("29876451_countVec.txt File generated")

29876451_countVec.txt File generated
